<a href="https://colab.research.google.com/github/zeinabmohamed/CIT690E-DeepLearning-Zeinab_Abdelmawla-191009/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1) Write a Python Colab Notebook: Use CIFAR-10 dataset and ResNet20, ResNet-32, ResNet-44 and ResNet-56 architectures in Pytorch to compute the
classification error and the number of parameters of each architecture as reported in
Table 6 of the following paper:
• He, Kaiming, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. "Deep residual learning for image
recognition." In Proceedings of the IEEE CVPR 2016.
• Please note you should train your models on CIFAR-10 training set from scratch using the
instructions given in the above paper and in particular Section 4.2 and Figure 3.

In [1]:
#modelResNet20 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet44', pretrained=True)
#modelResNet20.eval()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyper-parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001

Load the dataset CIFAR-10

In [ ]:
cifer10_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
cifer10_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

In [ ]:
train_dataloader = torch.utils.data.DataLoader(cifer10_trainset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(cifer10_testset, batch_size=batch_size, shuffle=False)

print("Training dataset size: ", len(cifer10_trainset))
print("Testing dataset size: ", len(cifer10_testset))

In [ ]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    

In [ ]:
def createResnetModelWithDepth(depth):
  noOfBlocks = int((depth-2/6))
  return ResNet(ResidualBlock, [noOfBlocks, noOfBlocks, noOfBlocks]).to(device)

In [ ]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# Train the model
import os

def trainModel(model,depth):
  # Loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  outdir ='./results'
  if not os.path.exists(outdir):
    os.mkdir(outdir)

  cols = ['epoch', 'train_loss', 'train_err', 'test_err']
  results_df = pd.DataFrame(columns=cols).set_index('epoch')
  
  total_step = len(train_dataloader)
  curr_lr = learning_rate
  results_file = f'results/resnet{depth}.csv'

  for epoch in range(num_epochs):
    running_loss  = 0.0
    best_test_err = 1.0

    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print average loss for last 50 mini-batches
        running_loss += loss.item()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
    
    train_loss = loss.item()
    train_err = evaluate(model, train_dataloader, device)
    test_err = evaluate(model, test_dataloader, device)
    results_df.loc[epoch] = [train_loss, train_err, test_err]
    results_df.to_csv(results_file)
    print(f'train_err: {train_err} test_err: {test_err}')

    model.eval()

  return train_loss


In [ ]:
def evaluate(model, data_loader, device):
    y_true = np.array([], dtype=np.int)
    y_pred = np.array([], dtype=np.int)
    
    with torch.no_grad():
      for data in data_loader:
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)

          y_true = np.concatenate((y_true, labels.cpu()))
          y_pred = np.concatenate((y_pred, predicted.cpu()))
    
    error = np.sum(y_pred != y_true) / len(y_true)
    return error


In [ ]:
# Test the model
def testModel(model):
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
#Calculate the total number of parameters for model stride 2
from prettytable import PrettyTable

def calculateModelParams(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
# Save the model checkpoint
def saveModel(model,layerNo):
  snapShotName = "resnet%s.ckpt" % (layerNo)
  torch.save(model.state_dict(), snapShotName)
  print(snapShotName)

**Resnet20**

In [ ]:
resnet20Model = createResnetModelWithDepth(20)
trainModel(resnet20Model,20)
testModel(resnet20Model)
saveModel(resnet20Model,20)

**Resnet32**

In [ ]:
resnet32Model = createResnetModelWithDepth(32)
trainModel(resnet32Model)
testModel(resnet32Model)
saveModel(resnet32Model,32)

**Resnet44**

In [ ]:
resnet44Model = createResnetModelWithDepth(44)
trainModel(resnet44Model)
testModel(resnet44Model)
saveModel(resnet44Model,44)

**Resnet56**

In [ ]:
resnet56Model = createResnetModelWithDepth(56)
trainModel(resnet56Model)
testModel(resnet56Model)
saveModel(resnet56Model,56)

In [ ]:
calculateModelParams(resnet20Model)
calculateModelParams(resnet32Model)
calculateMod9elParams(resnet44Model)
calculateModelParams(resnet56Model)